# YOLO11n-seg 訓練復現 Notebook

本 notebook 用於復現 `scripts/seg_workflow.sh` 中的手標資料訓練流程。
只需 **Run All** 即可完整復現訓練結果。

**對應的 shell 指令：**
```bash
python3 scripts/seg_train.py \
  --root /data3/TC/ros2_cooking_perception \
  --train \
  --train-data dataset_yolo/manual/dataset.yaml \
  --mask-method manual \
  --yolo-model yolo11n-seg.pt \
  --epochs 100 \
  --imgsz 320 \
  --batch 8 \
  --device 0 \
  --train-name-prefix seg \
  --train-exist-ok
```

**訓練配置：**
| 參數 | 值 |
|---|---|
| Model | `yolo11n-seg.pt` |
| Dataset | `dataset_yolo/manual/dataset.yaml` |
| Image size | 320 |
| Epochs | 100 |
| Batch size | 8 |
| Device | GPU 0 |

## 1. 設定參數

In [5]:
from pathlib import Path

# ── 訓練參數 (與 seg_workflow.sh 完全一致) ──
ROOT = Path("/data3/TC/ros2_cooking_perception")
DATASET_YAML = ROOT / "dataset_yolo" / "manual" / "dataset.yaml"
YOLO_MODEL = "yolo11n-seg.pt"
MASK_METHOD = "manual"
TRAIN_NAME_PREFIX = "seg"

EPOCHS = 100
IMGSZ = 320
BATCH = 8
DEVICE = "0"

# 訓練輸出目錄
PROJECT_DIR = ROOT / "runs" / "segment"
RUN_NAME = f"{TRAIN_NAME_PREFIX}_{MASK_METHOD}"

print("=== 訓練參數 ===")
print(f"  ROOT:          {ROOT}")
print(f"  DATASET_YAML:  {DATASET_YAML}")
print(f"  YOLO_MODEL:    {YOLO_MODEL}")
print(f"  EPOCHS:        {EPOCHS}")
print(f"  IMGSZ:         {IMGSZ}")
print(f"  BATCH:         {BATCH}")
print(f"  DEVICE:        {DEVICE}")
print(f"  OUTPUT:        {PROJECT_DIR / RUN_NAME}")

=== 訓練參數 ===
  ROOT:          /data3/TC/ros2_cooking_perception
  DATASET_YAML:  /data3/TC/ros2_cooking_perception/dataset_yolo/manual/dataset.yaml
  YOLO_MODEL:    yolo11n-seg.pt
  EPOCHS:        100
  IMGSZ:         320
  BATCH:         8
  DEVICE:        0
  OUTPUT:        /data3/TC/ros2_cooking_perception/runs/segment/seg_manual


## 2. 驗證資料集

In [6]:
import yaml

assert DATASET_YAML.exists(), f"Dataset YAML not found: {DATASET_YAML}"

with open(DATASET_YAML) as f:
    ds_cfg = yaml.safe_load(f)

print("=== dataset.yaml ===")
for k, v in ds_cfg.items():
    print(f"  {k}: {v}")

# 計算圖片數量
ds_root = Path(ds_cfg["path"])
train_dir = ds_root / ds_cfg["train"]
val_dir = ds_root / ds_cfg["val"]

n_train = len(list(train_dir.glob("*.png"))) if train_dir.exists() else 0
n_val = len(list(val_dir.glob("*.png"))) if val_dir.exists() else 0

print(f"\n=== 資料集大小 ===")
print(f"  Train: {n_train} images  ({train_dir})")
print(f"  Val:   {n_val} images  ({val_dir})")

assert n_train > 0, "Train images not found — 請先準備資料集"
assert n_val > 0, "Val images not found — 請先準備資料集"

=== dataset.yaml ===
  path: /data3/TC/ros2_cooking_perception/dataset_yolo/manual
  train: images/train
  val: images/val
  names: {0: 'black wok', 1: 'wooden handle spatula', 2: 'fried egg', 3: 'stainless steel bowl', 4: 'thermometer gun'}

=== 資料集大小 ===
  Train: 25 images  (/data3/TC/ros2_cooking_perception/dataset_yolo/manual/images/train)
  Val:   5 images  (/data3/TC/ros2_cooking_perception/dataset_yolo/manual/images/val)


## 3. 載入模型

In [8]:
from ultralytics import YOLO

model = YOLO(YOLO_MODEL)

print(f"Model: {YOLO_MODEL}")
print(f"Task:  {model.task}")
print(f"Classes: {model.names}")

Model: yolo11n-seg.pt
Task:  segment
Classes: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote',

## 4. 開始訓練

⚠️ **注意：** 100 epochs 約需 10–30 分鐘（視 GPU 而定）。
訓練結果會存放在 `runs/segment/seg_manual/`。

In [9]:
from datetime import datetime

print(f"開始訓練: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"輸出目錄: {PROJECT_DIR / RUN_NAME}")
print(f"exist_ok: True (若已存在會覆蓋)\n")

results = model.train(
    data=str(DATASET_YAML),
    imgsz=IMGSZ,
    epochs=EPOCHS,
    batch=BATCH,
    device=DEVICE,
    project=str(PROJECT_DIR),
    name=RUN_NAME,
    exist_ok=True,
)

print(f"\n訓練完成: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

開始訓練: 2026-02-09 22:29:38
輸出目錄: /data3/TC/ros2_cooking_perception/runs/segment/seg_manual
exist_ok: True (若已存在會覆蓋)

New https://pypi.org/project/ultralytics/8.4.13 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.4.11 🚀 Python-3.12.11 torch-2.10.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24081MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/data3/TC/ros2_cooking_perception/dataset_yolo/manual/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line

## 5. 取得訓練結果目錄

In [10]:
import pandas as pd

# 取得實際的 run directory
if hasattr(model, "trainer") and getattr(model.trainer, "save_dir", None) is not None:
    run_dir = Path(model.trainer.save_dir)
else:
    run_dir = PROJECT_DIR / RUN_NAME

print(f"Run directory: {run_dir}")
print(f"Best weights:  {run_dir / 'weights' / 'best.pt'}")
print(f"Last weights:  {run_dir / 'weights' / 'last.pt'}")

Run directory: /data3/TC/ros2_cooking_perception/runs/segment/seg_manual
Best weights:  /data3/TC/ros2_cooking_perception/runs/segment/seg_manual/weights/best.pt
Last weights:  /data3/TC/ros2_cooking_perception/runs/segment/seg_manual/weights/last.pt


## 6. 訓練曲線與指標分析

In [11]:
results_csv = run_dir / "results.csv"
assert results_csv.exists(), f"results.csv not found at {results_csv}"

df = pd.read_csv(results_csv)
# 清理欄位名稱 (ultralytics 有時帶空白)
df.columns = df.columns.str.strip()

print(f"共 {len(df)} epochs\n")

# 最後一個 epoch
print("=== 最後 epoch 指標 ===")
last = df.iloc[-1]
for col in df.columns:
    print(f"  {col}: {last[col]:.4f}" if isinstance(last[col], float) else f"  {col}: {last[col]}")

# 最佳 mAP50-95(B) epoch
map_col = "metrics/mAP50-95(B)"
if map_col in df.columns:
    best_idx = df[map_col].idxmax()
    print(f"\n=== 最佳 {map_col} (epoch {best_idx}) ===")
    best = df.loc[best_idx]
    for col in df.columns:
        print(f"  {col}: {best[col]:.4f}" if isinstance(best[col], float) else f"  {col}: {best[col]}")

共 100 epochs

=== 最後 epoch 指標 ===
  epoch: 100.0000
  time: 153.9310
  train/box_loss: 0.7573
  train/seg_loss: 1.1994
  train/cls_loss: 0.7860
  train/dfl_loss: 0.9457
  train/sem_loss: 0.0000
  metrics/precision(B): 0.9136
  metrics/recall(B): 0.9600
  metrics/mAP50(B): 0.9419
  metrics/mAP50-95(B): 0.7701
  metrics/precision(M): 0.9136
  metrics/recall(M): 0.9600
  metrics/mAP50(M): 0.9353
  metrics/mAP50-95(M): 0.6515
  val/box_loss: 0.7573
  val/seg_loss: 1.3257
  val/cls_loss: 0.5767
  val/dfl_loss: 0.8820
  val/sem_loss: 0.0000
  lr/pg0: 0.0000
  lr/pg1: 0.0000
  lr/pg2: 0.0000

=== 最佳 metrics/mAP50-95(B) (epoch 86) ===
  epoch: 87.0000
  time: 133.0910
  train/box_loss: 0.8105
  train/seg_loss: 1.2775
  train/cls_loss: 0.8696
  train/dfl_loss: 0.9420
  train/sem_loss: 0.0000
  metrics/precision(B): 0.9529
  metrics/recall(B): 0.9600
  metrics/mAP50(B): 0.9553
  metrics/mAP50-95(B): 0.8054
  metrics/precision(M): 0.9529
  metrics/recall(M): 0.9600
  metrics/mAP50(M): 0.9553
  me

## 7. 訓練曲線圖

In [12]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Box mAP
ax = axes[0, 0]
if "metrics/mAP50(B)" in df.columns:
    ax.plot(df["metrics/mAP50(B)"], label="mAP50(B)")
if "metrics/mAP50-95(B)" in df.columns:
    ax.plot(df["metrics/mAP50-95(B)"], label="mAP50-95(B)")
ax.set_title("Box mAP")
ax.set_xlabel("Epoch")
ax.legend()
ax.grid(True, alpha=0.3)

# Seg mAP
ax = axes[0, 1]
if "metrics/mAP50(M)" in df.columns:
    ax.plot(df["metrics/mAP50(M)"], label="mAP50(M)")
if "metrics/mAP50-95(M)" in df.columns:
    ax.plot(df["metrics/mAP50-95(M)"], label="mAP50-95(M)")
ax.set_title("Mask (Seg) mAP")
ax.set_xlabel("Epoch")
ax.legend()
ax.grid(True, alpha=0.3)

# Box loss
ax = axes[1, 0]
for col in df.columns:
    if "box_loss" in col or "cls_loss" in col or "dfl_loss" in col:
        if "train" in col:
            ax.plot(df[col], label=col.split("/")[-1])
ax.set_title("Train Losses")
ax.set_xlabel("Epoch")
ax.legend()
ax.grid(True, alpha=0.3)

# Seg loss
ax = axes[1, 1]
for col in df.columns:
    if "seg_loss" in col:
        ax.plot(df[col], label=col.split("/")[-1])
ax.set_title("Seg Loss")
ax.set_xlabel("Epoch")
ax.legend()
ax.grid(True, alpha=0.3)

fig.suptitle(f"YOLO11n-seg Training — {MASK_METHOD} labels", fontsize=14)
fig.tight_layout()
plt.show()

<Figure size 1400x1000 with 4 Axes>

## 8. 驗證 (Validation)

用訓練好的 best.pt 在 val set 上跑一次完整驗證。

In [13]:
best_pt = run_dir / "weights" / "best.pt"
assert best_pt.exists(), f"best.pt not found: {best_pt}"

val_model = YOLO(str(best_pt))
val_results = val_model.val(data=str(DATASET_YAML), device=DEVICE)

print("=== Validation Results ===")
print(f"  Box  mAP50:    {val_results.box.map50:.4f}")
print(f"  Box  mAP50-95: {val_results.box.map:.4f}")
if hasattr(val_results, "seg") and val_results.seg is not None:
    print(f"  Seg  mAP50:    {val_results.seg.map50:.4f}")
    print(f"  Seg  mAP50-95: {val_results.seg.map:.4f}")

# Per-class results
if hasattr(val_results.box, "ap50"):
    class_names = val_model.names
    print("\n=== Per-class Box AP50 ===")
    for i, ap in enumerate(val_results.box.ap50):
        print(f"  {class_names.get(i, i):30s}  AP50={ap:.4f}")

Ultralytics 8.4.11 🚀 Python-3.12.11 torch-2.10.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24081MiB)
YOLO11n-seg summary (fused): 114 layers, 2,835,543 parameters, 0 gradients, 9.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1367.9±228.4 MB/s, size: 165.4 KB)
val: Scanning /data3/TC/ros2_cooking_perception/dataset_yolo/manual/labels/val.cache... 5 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 5/5 911.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 2.8it/s 0.4s
                   all          5         23      0.953       0.96      0.955      0.805      0.953       0.96      0.955      0.652
             black wok          5          5      0.948          1      0.995      0.886      0.948          1      0.995      0.876
 wooden handle spatula          5          5      0.983          1      0.995      0.939      0.983          1      0.995      0.535
